In [159]:
# IMPORTING LIBRARIES 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier 
import sklearn.datasets
import sklearn.preprocessing
import sklearn.random_projection
import sklearn.neighbors

In [1]:
%run data/preprocessing.py

NameError: name 'pd' is not defined

In [160]:
# starting with these parameters
n = 3
profit_taking = 0.007
p = 0.68

In [161]:
# READ IN DATA
df_original["Date"] = pd.to_datetime(df_original["Date"], format="%m/%d/%y")

df_original = df_original.sort_values(by =  "Date", ascending=True)

df_original = df_original.rename(columns = {'Price': 'Close'})
df_original = df_original[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']]
df = df_original[['Date', 'Close', 'Open', 'High', 'Low', 'Change %']].copy()
df.head()

,Date,Close,Open,High,Low,Change %
2629,2005-09-16,0.6765,0.6768,0.6790,0.6746,0.00%
2628,2005-09-19,0.6735,0.6747,0.6759,0.6723,-0.44%
2627,2005-09-20,0.6736,0.6734,0.6753,0.6725,0.01%
2626,2005-09-21,0.6750,0.6737,0.6763,0.6727,0.21%
2625,2005-09-22,0.6786,0.6750,0.6800,0.6743,0.53%


In [162]:
# CREATE PRIOR n DAYS FEATURE

for before in range(1, n+1):
    df[f"Close_{before}_before"] = df["Close"].shift(before)
    df[f"Open_{before}_before"] = df["Open"].shift(before)
    df[f"High_{before}_before"] = df["High"].shift(before)
    df[f"Low_{before}_before"] = df["Low"].shift(before)

df["target"] = (df["High"] > (profit_taking+1)*df["Open"]).astype(int)
target = df[["target"]]
df = df.dropna()

In [163]:
train_df   = df[(df['Date'] >= '2016-01-01') & (df['Date'] < '2020-01-01')]
val_df   = df[(df['Date'] >= '2020-01-01') & (df['Date'] < '2022-01-01')]
test_df  = df[df['Date'] >= '2022-01-01']

x_train = train_df.drop(columns=['target', 'Date', 'Change %', 'Close', 'Open', 'High', 'Low'])
y_train = train_df['target']

x_val = val_df.drop(columns=['target', 'Date', 'Change %', 'Close', 'Open', 'High', 'Low'])
y_val = val_df['target']

x_test = test_df.drop(columns=['target', 'Date', 'Change %', 'Close', 'Open', 'High', 'Low'])
y_test = test_df['target']

In [164]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))

(1043, 12)
(523, 12)
(1011, 12)
target
0    0.825503
1    0.174497
Name: proportion, dtype: float64
target
0    0.866157
1    0.133843
Name: proportion, dtype: float64


In [165]:
# Scaling the data
scaler = StandardScaler()
scaler.fit(x_train)   

x_train = scaler.transform(x_train)
x_val   = scaler.transform(x_val)
x_test  = scaler.transform(x_test)


In [166]:
# MODEL TRAINING
model1 = MLPClassifier(solver='adam',
                       alpha=1e-5,
                       hidden_layer_sizes=(100,), 
                       random_state=1)

model1.fit(x_train, y_train)

validation_accuracy = model1.score(x_val, y_val)
print(f"validation_accuracy1={validation_accuracy:0.4f}")
train_accuracy = model1.score(x_train, y_train)
print(f"train_accuracy1={train_accuracy:0.4f}")

    
model2 = sklearn.neighbors.KNeighborsClassifier(
    n_neighbors=4,
    weights='uniform',
    algorithm='ball_tree',
    )

model2.fit(x_train, y_train)

validation_accuracy = model2.score(x_val, y_val)
print(f"validation_accuracy2={validation_accuracy:0.4f}")
train_accuracy = model2.score(x_train, y_train)
print(f"train_accuracy2={train_accuracy:0.4f}")

validation_accuracy1=0.8662
train_accuracy1=0.8255
validation_accuracy2=0.8566
train_accuracy2=0.8322


In [167]:
if True:
    test_accuracy = model2.score(x_test, y_test)
    print(f"test_accuracy={test_accuracy}")

    y_true = y_test.values.ravel()
    probs_test = model2.predict_proba(x_test)[:, 1]
    preds_test = model2.predict(x_test)

    test_accuracy = model1.score(x_test, y_test)
    print(f"test_accuracy={test_accuracy}")

    y_true = y_test.values.ravel()
    probs_test = model1.predict_proba(x_test)[:, 1]
    preds_test = model1.predict(x_test)

test_accuracy=0.8585558852621167
test_accuracy=0.913946587537092


In [168]:
x_test = test_df.drop(columns=['target','Date','Change %','Close','Open','High','Low'])

x_test_scaled = scaler.transform(x_test)

df_evaluation = test_df.copy()

df_evaluation['prob'] = model2.predict_proba(x_test_scaled)[:, 1]

df_evaluation['enter'] = (df_evaluation['prob'] >= p).astype(int)

df_evaluation['pnl_if_hit']  = profit_taking
df_evaluation['pnl_if_miss'] = 0.0

df_evaluation['pnl'] = np.where(
    df_evaluation['enter'] == 1,
    np.where(df_evaluation['target'] == 1,
             df_evaluation['pnl_if_hit'],
             df_evaluation['pnl_if_miss']),
    0.0
)

In [169]:
if True:
    total_return = df_evaluation["pnl"].sum()
    num_trades = df_evaluation["enter"].sum()
    hit_rate = df_evaluation.loc[df_evaluation["enter"] == 1, "target"].mean()
    avg_return_per_trade = df_evaluation.loc[df_evaluation["enter"] == 1, "pnl"].mean()

    #print("Total return:", total_return)
    print("Number of trades:", num_trades)
    print("Hit rate:", hit_rate)
    print("Avg return per trade:", avg_return_per_trade)

Number of trades: 74
Hit rate: 0.12162162162162163
Avg return per trade: 0.0008513513513513513


In [170]:
df_evaluation['prob'].describe()

count    1011.000000
mean        0.229228
std         0.234286
min         0.000000
25%         0.000000
50%         0.250000
75%         0.500000
max         0.750000
Name: prob, dtype: float64

In [146]:
#eval NN

x_test = test_df.drop(columns=['target','Date','Change %','Close','Open','High','Low'])

x_test_scaled = scaler.transform(x_test)

df_evaluation = test_df.copy()

df_evaluation['prob'] = model1.predict_proba(x_test_scaled)[:, 1]

df_evaluation['enter'] = (df_evaluation['prob'] >= p).astype(int)

df_evaluation['pnl_if_hit']  = profit_taking
df_evaluation['pnl_if_miss'] = 0.0

df_evaluation['pnl'] = np.where(
    df_evaluation['enter'] == 1,
    np.where(df_evaluation['target'] == 1,
             df_evaluation['pnl_if_hit'],
             df_evaluation['pnl_if_miss']),
    0.0
)

total_return = df_evaluation["pnl"].sum()
num_trades = df_evaluation["enter"].sum()
hit_rate = df_evaluation.loc[df_evaluation["enter"] == 1, "target"].mean()
avg_return_per_trade = df_evaluation.loc[df_evaluation["enter"] == 1, "pnl"].mean()

perfect_trader = df.evaluations

#print("Total return:", total_return)
print("Number of trades:", num_trades)
print("Hit rate:", hit_rate)
print("Avg return per trade:", avg_return_per_trade)

Number of trades: 1011
Hit rate: 0.6904055390702275
Avg return per trade: 0.001035608308605341


In [148]:
df_evaluation.head()

,Date,Close,Open,High,Low,Change %,Close_1_before,Open_1_before,High_1_before,Low_1_before,...,Close_3_before,Open_3_before,High_3_before,Low_3_before,target,prob,enter,pnl_if_hit,pnl_if_miss,pnl
3640,2022-01-03,0.8381,0.8403,0.8421,0.8371,-0.23%,0.8400,0.8391,0.8418,0.8367,...,0.8412,0.8419,0.8426,0.8402,1,0.865998,1,0.0015,0.0,0.0015
3639,2022-01-04,0.8339,0.8386,0.8396,0.8335,-0.50%,0.8381,0.8403,0.8421,0.8371,...,0.8388,0.8414,0.8419,0.8378,0,0.866927,1,0.0015,0.0,0.0000
3638,2022-01-05,0.8344,0.8342,0.8374,0.8334,0.06%,0.8339,0.8386,0.8396,0.8335,...,0.8400,0.8391,0.8418,0.8367,1,0.871802,1,0.0015,0.0,0.0015
3637,2022-01-06,0.8342,0.8346,0.8374,0.8341,-0.02%,0.8344,0.8342,0.8374,0.8334,...,0.8381,0.8403,0.8421,0.8371,1,0.876108,1,0.0015,0.0,0.0015
3636,2022-01-07,0.8357,0.8346,0.8367,0.8336,0.18%,0.8342,0.8346,0.8374,0.8341,...,0.8339,0.8386,0.8396,0.8335,1,0.870218,1,0.0015,0.0,0.0015
